In [73]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import statsmodels.api as sm
import warnings

In [74]:
# def plot(file):
#     df=pd.read_csv('./../data/composite_vars/' + file)
#     df['date'] = pd.to_datetime(df['date'])
#     df.set_index('date', inplace=True)
#     plt.figure(figsize=(14, 7))
#     plt.style.use('dark_background')
#     plt.title(file)
#     plt.plot(df.index, df[df.columns[0]])
#     plt.show()

df=pd.read_csv('./../data/positive_composite/NIM.csv')
#arr = ['LAS','NIA','OE','PR','ROA','SIZE']
# arr = ['LAS']
arr = ['CDR', 'CR','INF',
       'NIA','OE','PR', 'RA',
       'ROA','SCTA','SIZE']
for name in arr:
    gotten=pd.read_csv('./../data/positive_composite/'+name+".csv")
    df[gotten.columns[1]]=gotten[gotten.columns[1]]
# print(df)
Y=df['NIM']
X=df[arr]
model = sm.OLS(Y, X).fit()

print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                    NIM   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                              1671.
Date:                Mon, 29 Apr 2024   Prob (F-statistic):                    3.40e-50
Time:                        21:57:29   Log-Likelihood:                          349.67
No. Observations:                  51   AIC:                                     -679.3
Df Residuals:                      41   BIC:                                     -660.0
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------